In [122]:
# creates new directory with con image, t-maps, z-maps, and temporal BOLD 4D images
# and design files for OpenAutism (per subject, per task, per run)
#only copy over files on the spreadsheet (/om/user/rezzo/EXPERIMENT_INFO/tomloc_subject_info_internal.csv)

# FOR FUTURE / PENDING:
# once second level is run, must import one con, t-map and z-map per subject (now importing 
# one per run)

# some subjects are in more than one experiment (e.g. SAX_OA_## in both exp1
# and exp2). In these cases, all runs are copied over. There is a second option, by using the function
# identify_greater_motion, which returns a list of the old subjectids to be eliminated. Based framewise displacement. 

pilot = 1        # 1 = pilot_data / 0 = analysis_data

# import modules
from glob import glob
import re
import os.path
from itertools import repeat
import csv
from shutil import copyfile
import shutil
import random
import numpy as np
import pandas as pd
from collections import Counter

if pilot == 1:
    analy_dir = 'pilot_data'
    conversion = '/om/user/rezzo/TOMLOC_INFO/tomloc_subject_info_internal.csv'
    datainfo = pd.read_csv('/om/user/rezzo/TOMLOC_INFO/Data_sets_internal.csv')
    details = pd.read_csv('/om/user/rezzo/TOMLOC_INFO/Description_of_functional_tasks_internal.txt', delimiter='\t')
    runinfo = '/om/group/saxelab/OpenAutism/TIER_OpenAutism/pilot_data/RUN_INFO/'
else:
    analy_dir = 'analysis_data' # note: make sure these csv files do not have special characters (e.g. ä)
    conversion = '/om/group/saxelab/OpenAutism/data/Subject_Task_Info_Dima/subject_info_internal.csv'
    datainfo = '/om/group/saxelab/OpenAutism/data/Subject_Task_Info_Dima/Data_sets_internal.csv'
    details = '/om/group/saxelab/OpenAutism/data/Subject_Task_Info_Dima/Description_of_functional_tasks_external.csv'
    runinfo = '/om/group/saxelab/OpenAutism/TIER_OpenAutism/analysis_data/RUN_INFO/'

In [108]:
def Obtain_OA_list(tmp_root):
    all_OA = []
    with open(tmp_root, "r") as tsv:
        for line in csv.reader(tsv,  delimiter = ","):
            all_OA.append(line[0])
    return all_OA

In [109]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [110]:
# optional function to use, if decide to remove the experiment of duplicate subject based 
# on greater motion.

def identify_greater_motion(subject1, subject2):
    tmp_root = '/om/user/rezzo/INCLUDE_final.csv'

    with open(tmp_root, "r") as tsv:
        mot1 = []
        mot2 = []
        subjectarray = [subject1, subject2]
        readCSV = csv.reader(tsv, delimiter='\t')
        for line in readCSV:
            if subject1 == line[2]:
                mot1.append(float(line[9]))
                np.array(mot1)
            if subject2 == line[2]:
                mot2.append(float(line[9]))
                np.array(mot2)
    if np.mean(mot1) > np.mean(mot2):
        return subject1
    elif np.mean(mot2) > np.mean(mot1):
        return subject2
    else:
        return random.choice(subjectarray) # for temporarily missing data, just choose randomly

In [111]:
def Convert_2int(runs):
    new_runs = [x.strip() for x in runs.split(', ')]
    counter = -1
    for el in new_runs:
        counter = counter+1
        new_runs[counter] = int(el)
    return new_runs

In [112]:
def remove_sub(name):
    if name[:4] == 'sub-':
        name = name[4:]
    return name

In [113]:
# subject look up table conversion (IGNORING undescores)
def Convert_Subname(Oldname):
    tmp_root = '/om/user/rezzo/Subject_Conversion_Table.csv'

    with open(tmp_root, "r") as tsv:
        for line in csv.reader(tsv,  delimiter = ","):
            if Oldname == line[1].replace("_",""):
                Newname = line[0]
            else:
                continue
    return Newname  

In [114]:
def Remove_Underscores(list_of_names):
    counter = 0
    for el in list_of_names:
        list_of_names[counter] = el.replace("_","")
        counter = counter + 1
    return list_of_names

In [115]:
def Return_dataset(site, task, details, datainfo):    
    dataset = datainfo.loc[datainfo['study_name'] == site, 'data_set'].iloc[0]
    dfList = details['data_set'].tolist()
    app = []
    for idx, el in enumerate(dfList):
        numerate = [int(s) for s in el.split(',')]
        if int(dataset) in numerate:
            if details.get_value(idx,'task_name') == task:
                return details.get_value(idx,'TR (sec)'), details.get_value(idx,'condition_1'), details.get_value(idx,'event or block design')
    

In [125]:
# # Looks for subjects that participated in > 1 experiment

df = pd.read_csv(conversion, header=None)

# make list of all duplicate items
a = df[0].tolist()
dup_list = [k for k,v in Counter(a).items() if v>1]

repeat_subjects = []
remove_subjects = []

for el in dup_list:
    with open(conversion, "r") as tsv:
        for line in csv.reader(tsv,  delimiter = ","):
            if el == line[0]:
                repeat_subjects.append(line[1])

list_of_duplicates = [repeat_subjects[i:i + 2] for i in range(0, len(repeat_subjects), 2)]

In [117]:
# def Return_motion(subject, site, run, motioninfo):   
def Return_motion(subject, site, task, run, motioninfo):   
    res = motioninfo.loc[(motioninfo[1] == site) & (motioninfo[0] == subject) &(motioninfo[3] == task) & (motioninfo[4] == run)]
    exclude = int(res[8])
    avemotion = float(res[9])
    return exclude, avemotion

In [143]:
root = '/om/group/saxelab/OpenAutism/Analysis/first_level_standard.py/'
OA_dir = '/om/group/saxelab/OpenAutism/TIER_OpenAutism/'+analy_dir+'/SUBJECTS/'
path = '/om/user/rezzo/INCLUDE.csv'
motioninfo = pd.read_csv(path, sep='\t', header=None)

df = pd.read_csv(conversion)
df
dup_list = []
UNIQUE_OA_IDS = df['OAID'].unique()
UNIQUE_TASKS = df['func_task'].unique()

# for each unique OA subject, return row
for newname in UNIQUE_OA_IDS:
    
    if pilot == 1:
        UNIQUE_OA_IDS = [ task for task in UNIQUE_OA_IDS if "tomloc" in task ]
    else:
        UNIQUE_OA_IDS = [ task for task in UNIQUE_OA_IDS if "tomloc" not in task ]

        #create OA directory in TIER folder is non-existent
    if (not os.path.exists(OA_dir+newname)):

        proc_streams = ['first_level_standard.py','first_level_aroma.py','first_level_gorgolewski.py']
        time_streams = ['first_level_standard_timeseries.py','first_level_aroma_timeseries.py','first_level_gorgolewski_timeseries.py']

        for stream in range(0, len(proc_streams)):

            stream_abrv = find_between(proc_streams[stream], "level_", ".py" )

            # make directories
            os.makedirs(os.path.join(OA_dir+newname,stream_abrv,'first_level_analyses','BOLD_data')) 
            os.makedirs(os.path.join(OA_dir+newname,stream_abrv,'second_level_analyses','multivariate'))
            os.makedirs(os.path.join(OA_dir+newname,stream_abrv,'second_level_analyses','lateralization'))
            os.makedirs(os.path.join(OA_dir+newname,stream_abrv,'second_level_analyses','magnitude'))
            os.makedirs(os.path.join(OA_dir+newname,stream_abrv,'second_level_analyses','interregional_cor'))
            os.makedirs(os.path.join(OA_dir+newname,stream_abrv,'second_level_analyses','temporal_variance'))
            os.makedirs(os.path.join(OA_dir+newname,stream_abrv,'second_level_analyses','MISC','individual_roi_masks'))
            os.makedirs(os.path.join(OA_dir+newname,stream_abrv,'second_level_analyses','MISC','mean_roi_Temporal_Signal'))

            for task in UNIQUE_TASKS:
                # create unique rows
                dup_list = []
                dup_list.append(df.index[(df['OAID'] == newname) & (df['func_task'] == task)].tolist())
                task = Remove_Underscores([task])
                task = task[0]
                run_counter = 0
                
                for items in dup_list: # each item is the indices list of all runs for a task (across experiments)
                    site = df.iloc[items]['data_set'].astype(str)
                    site = site.tolist() #site
                    found = df.iloc[items]['SUBJID'].astype(str)
                    found = Remove_Underscores(found.tolist()) #subjid
                    
                    for el in range(0,len(items)): 
                        items_index = items[el]
                        runs = df.iloc[items_index]['BIDS_runs']
                        runs = Convert_2int(runs)
                        
                        for run in runs:
                            
                            run_counter = run_counter+1
                            
                            #added
                            if stream == 0:
                                #SAVE MOTION INTO
                                [exclude, avemotion] = Return_motion(newname,site[el], task, run, motioninfo)
                                
                                # SAVE INFO NEEDED IN METRIC SCRIPT (needed b/c subjects in multiple experiments)
                                [TR, COND, BLOCK] = Return_dataset(site[el], task, details, datainfo)
                                with open(runinfo+"RUN_INFO.txt",'a') as fd:
                                    fd.write(newname + "\t" + site[el] + "\t" + task + "\t" + str(run_counter) + "\t" + str(TR) + "\t" + str(COND) + "\t" + str(BLOCK) +"\t" +str(exclude)+"\t" +str(avemotion) +"\n")
                                #fd.close()
                                
                                # look for any runs that have to be manually deleted due to scanner issue
                                originalbidruns = '/om/group/saxelab/OpenAutism/data/'+site[el]+'/BIDS/'+'sub-'+found[el]+'/func/'
                                bidruns = [filename for filename in sorted(os.listdir(originalbidruns)) if (filename.startswith('sub-'+found[el]) and filename.endswith(".nii.gz"))]
                                amount = len(bidruns)
                                if amount != len(runs):
                                    print(found[el]': BID funcs > valid runs. May reqcuire manual deletion.')
                                
                            src_dir = '/om/group/saxelab/OpenAutism/Analysis/'+proc_streams[stream]+'/'+site[el]+'/sub-'+found[el]+'/'+task+'/model/run'+str(run)+'/'
                            design_dir = '/om/group/saxelab/OpenAutism/data/TextFiles/tsv_text_files/'+site[el]+'/'
                            raw_dir = "/om/group/saxelab/OpenAutism/Analysis/"+time_streams[stream]+'/'+site[el]+"/sub-"+found[el]+'/'+task+"/model/run"+str(run)+'/'
                            
                            # if the run is not a motion outlier:
                            
                            try:
                                #con, tmap, zmap
                                prefixed1 = [filename for filename in sorted(os.listdir(src_dir)) if (filename.startswith("con_1_") and filename.endswith("_tstat.nii.gz"))]
                                copyfile(src_dir+prefixed1[0], OA_dir+newname+'/'+stream_abrv+'/first_level_analyses/tstat1_'+task+'_run-00'+str(run_counter)+'.nii.gz')
                                prefixed2 = [filename for filename in sorted(os.listdir(src_dir)) if (filename.startswith("con_1_") and filename.endswith("_cope.nii.gz"))]
                                copyfile(src_dir+prefixed2[0], OA_dir+newname+'/'+stream_abrv+'/first_level_analyses/cope1_'+task+'_run-00'+str(run_counter)+'.nii.gz')
                                prefixed3 = [filename for filename in sorted(os.listdir(src_dir)) if (filename.startswith("con_1_") and filename.endswith("_zstat.nii.gz"))]
                                copyfile(src_dir+prefixed3[0], OA_dir+newname+'/'+stream_abrv+'/first_level_analyses/zstat1_'+task+'_run-00'+str(run_counter)+'.nii.gz')
                            except FileNotFoundError:
                                print('WARNING: missing image file(s) in '+src_dir)
                                
                            try:
                                #temporal runs
                                prefixed4 = [filename for filename in sorted(os.listdir(raw_dir)) if (filename.startswith("res4d") and filename.endswith(".nii.gz"))]
                                copyfile(raw_dir+prefixed4[0], OA_dir+newname+'/'+stream_abrv+'/first_level_analyses/BOLD_data/temporaldata_'+task+'_run-00'+str(run_counter)+'.nii.gz')
                            except FileNotFoundError:
                                print('WARNING: missing temopral file(s) in '+raw_dir)
                                
                            try:
                                #design file
                                copyfile(design_dir+'sub-'+found[el]+'_task-'+task+'_run-00'+str(run)+'_events.tsv', OA_dir+newname+'/'+stream_abrv+'/first_level_analyses/BOLD_data/design_'+task+'_run-00'+str(run_counter)+'.tsv') 
                            except FileNotFoundError:
                                print('WARNING: missing design file(s) in '+design_dir)

/opt/conda/envs/neuro/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/opt/conda/envs/neuro/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


Runs match
Runs match


In [135]:
# pull in file with all run info and make a file listing only runs to include

if pilot == 1:
    con = 'pilot_data'
else:
    con = 'analysis_data'
    
runinfo = '/om/group/saxelab/OpenAutism/TIER_OpenAutism/'+con+'/RUN_INFO/RUN_INFO.txt'

Motion_filt_lenient = []

with open(runinfo, "r") as infile:
    X = [list(map(str, line.split())) for line in infile]
    for line in X:
        if line[7] == '0':
            Motion_filt_lenient.append(line)
        
# save to a text file
fo = open("/om/group/saxelab/OpenAutism/TIER_OpenAutism/"+con+"/RUN_INFO/Runs_LenientMotionFiltered.tsv", "w+")
for element in range (0,len(Motion_filt_lenient)):
    fo.write(Motion_filt_lenient[element][0]+ Motion_filt_lenient[element][2]+'00'+Motion_filt_lenient[element][3]+ "\n")
fo.close()
